In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Flatten , Dense , Dropout , Activation
from keras.layers.convolutional import Convolution2D , MaxPooling2D , ZeroPadding2D
from keras.losses import categorical_crossentropy
from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import SGD,Adam
import os
from sklearn.model_selection import train_test_split
os.chdir("C:/Users/Ruchita/Desktop/Project_Dataset/")

In [2]:
age_gender_data = pd.read_csv('age_gender.csv')

In [3]:
age_gender_data

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...
...,...,...,...,...,...
23700,99,0,1,20170120221920654.jpg.chip.jpg,127 100 94 81 77 77 74 99 102 98 128 145 160 1...
23701,99,1,1,20170120134639935.jpg.chip.jpg,23 28 32 35 42 47 68 85 98 103 113 117 130 129...
23702,99,2,1,20170110182418864.jpg.chip.jpg,59 50 37 40 34 19 30 101 156 170 177 184 187 1...
23703,99,2,1,20170117195405372.jpg.chip.jpg,45 108 120 156 206 197 140 180 191 199 204 207...


In [4]:
pixels = []
age = []

for index , row in age_gender_data.iterrows():
    k = row['pixels'].split(" ")
    pixels.append(np.array(k , 'float32'))
    p = row['age']
    if(p > 0 and p <= 18):
        age.append(0)
    elif(p > 18 and p <= 45):
        age.append(1)
    elif(p > 45 and p <= 60):
        age.append(2)
    else:
        age.append(3)

pixels_train = []
age_train = []
pixels_test = []
age_test = []

pixels_train , pixels_test , age_train, age_test = train_test_split(pixels , age , test_size=0.2 , random_state = 42)

In [5]:
pixels_train = np.array(pixels_train , 'float32')
pixels_test = np.array(pixels_test , 'float32')
Age_train = np.array(age_train , 'float32')
Age_test = np.array(age_test , 'float32')

pixels_train = pixels_train.reshape(pixels_train.shape[0] , 48 , 48 , 1)
pixels_test = pixels_test.reshape(pixels_test.shape[0] , 48 , 48 , 1)

age_train = to_categorical(age_train , num_classes = 4)
age_test = to_categorical(age_test , num_classes = 4)

In [6]:
model = Sequential()

model.add(Convolution2D(input_shape = (48 , 48 , 1) , filters = 64 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 64 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 128 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 128 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 256 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 256 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 256 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(Convolution2D(filters = 512 , kernel_size = (3 , 3) , padding = 'same' , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2 , 2) , strides = (2 , 2)))

model.add(Flatten())
model.add(Dense(units = 4096 , activation = 'relu'))
model.add(Dense(units = 4096 , activation = 'relu'))

model.add(Dense(units = 4 , activation = 'softmax'))

In [7]:
model.compile(optimizer = 'sgd' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [8]:


model.fit(pixels_train , age_train , batch_size = 64 , epochs = 30 , verbose = 1 , validation_data = (pixels_test , age_test) , validation_split=0.2)

Epoch 1/30
238/238 [==============================] - 686s 3s/step - loss: 1.1272 - accuracy: 0.5750 - val_loss: 2.2902 - val_accuracy: 0.5597
Epoch 2/30
238/238 [==============================] - 662s 3s/step - loss: 1.0128 - accuracy: 0.6147 - val_loss: 1.4125 - val_accuracy: 0.3414
Epoch 3/30
238/238 [==============================] - 589s 2s/step - loss: 0.8623 - accuracy: 0.6731 - val_loss: 1.1546 - val_accuracy: 0.5634
Epoch 4/30
238/238 [==============================] - 584s 2s/step - loss: 0.7842 - accuracy: 0.7029 - val_loss: 0.9927 - val_accuracy: 0.6093
Epoch 5/30
238/238 [==============================] - 587s 2s/step - loss: 0.7213 - accuracy: 0.7276 - val_loss: 1.4264 - val_accuracy: 0.4516
Epoch 6/30
238/238 [==============================] - 651s 3s/step - loss: 0.6833 - accuracy: 0.7394 - val_loss: 0.9713 - val_accuracy: 0.6151
Epoch 7/30
238/238 [==============================] - 617s 3s/step - loss: 0.6389 - accuracy: 0.7535 - val_loss: 1.7040 - val_accuracy: 0.2207

In [9]:
loss_and_metrics = model.evaluate(pixels_test , age_test)
print(loss_and_metrics)

149/149 [==============================] - 33s 219ms/step - loss: 0.9590 - accuracy: 0.7891
[0.9589807987213135, 0.7890740633010864]


In [10]:
model_json = model.to_json()
with open("model_age.json" , "w") as json_file:
    json_file.write(model_json)
    model.save_weights("model_age.h5")
    print("Saved model to disk")

Saved model to disk
